In [2]:
# Modeling using TITLE instead of content

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from nltk.tokenize import sent_tokenize
import collections, re
from sklearn import preprocessing, cross_validation, svm
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from keras.preprocessing.text import Tokenizer
import re
import tensorflow as tf

/Users/mollyperlich/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/mollyperlich/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
#read data
data = pd.read_json('2.News_With_Labels.json')
data.head()

,createdate,articles
0,2019-07-04,"{'publishdate': '1/2/2018', 'content': ' Mor..."
1,2019-07-04,"{'publishdate': '1/2/2018', 'content': 'If Pro..."
2,2019-07-04,"{'publishdate': '1/2/2018', 'content': 'A YouT..."
3,2019-07-04,"{'publishdate': '1/2/2018', 'content': ' Mor..."
4,2019-07-04,"{'publishdate': '1/2/2018', 'content': 'Q. I ..."


In [4]:
#convert to df
df = pd.DataFrame(data)
df.head()

,createdate,articles
0,2019-07-04,"{'publishdate': '1/2/2018', 'content': ' Mor..."
1,2019-07-04,"{'publishdate': '1/2/2018', 'content': 'If Pro..."
2,2019-07-04,"{'publishdate': '1/2/2018', 'content': 'A YouT..."
3,2019-07-04,"{'publishdate': '1/2/2018', 'content': ' Mor..."
4,2019-07-04,"{'publishdate': '1/2/2018', 'content': 'Q. I ..."


In [6]:
stock_df = pd.DataFrame([])

for i in df['articles']:
    stock_df = stock_df.append(pd.DataFrame({'title': i['title'], 'label': i['label']}, index=[0]), ignore_index=True)

print(f"Number of words in raw data: {stock_df['title'].apply(lambda x: len(x.split(' '))).sum()}") 
stock_df.head()

Number of words in raw data: 48384


,title,label
0,How an A.I. ‘Cat-and-Mouse Game’ Generates Bel...,Buy
1,Activists’ Guide to 2018: The Best Defense Is ...,Buy
2,"Logan Paul, YouTube Star, Says Posting Video o...",Buy
3,How to Manage Your Career,Buy
4,Adjusting Twitter’s Sensitivities,Buy


In [7]:
#keep a count of how many you delete
bad_rows = []
for i in range(0,stock_df.shape[0]):
    #if the number of characters is below 100
    if len(stock_df['title'].iloc[i])<100:
        bad_rows.append(i)

In [10]:
import nltk
nltk.download('stopwords')

#clean up the text
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

#function to clean the text
def clean_text(text):
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwords from text
    return text
    
#apply function to each row in the dataframe
stock_df['title'] = stock_df['title'].apply(clean_text)
#number of words after clean up
print(f"Number of words AFTER clean up: {stock_df['title'].apply(lambda x: len(x.split(' '))).sum()}") 
stock_df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mollyperlich/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Number of words AFTER clean up: 34487


,title,label
0,ai catandmouse game generates believable fake ...,Buy
1,activists guide 2018 best defense good offense,Buy
2,logan paul youtube star says posting video dea...,Buy
3,manage career,Buy
4,adjusting twitters sensitivities,Buy


In [11]:
content_ = []
for item in stock_df["title"]:
    if (item in content_):
        continue
    else:
        content_.append(item)

In [12]:
from sklearn.model_selection import train_test_split

X = stock_df.title
y = stock_df.label
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state = 42)

In [13]:
bagsofwords = [ collections.Counter(re.findall(r'\w+', txt))
            for txt in content_]
bagsofwords[0]

Counter({'ai': 1,
         'catandmouse': 1,
         'game': 1,
         'generates': 1,
         'believable': 1,
         'fake': 1,
         'photos': 1})

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(content_) 
print(X.toarray())
print(vectorizer.get_feature_names())

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
['000', '004', '10', '100', '101', '108', '11', '110', '113', '114', '119', '12', '120', '123', '126', '128', '13', '135', '1380', '14', '146', '148', '15', '150', '153', '15year', '16', '17', '173', '178', '18', '1800gotjunk', '1897', '19', '195', '196', '1960s', '1968', '1969', '1980s', '1990s', '1mdb', '20', '200', '2006', '2009', '2011', '2012', '2014', '2015', '2016', '2017', '2017s', '2018', '2018s', '2019', '2020', '2040', '20s', '20th', '21', '214', '215', '218', '21st', '21year', '22', '23', '238', '24', '248', '249', '24yearold', '25', '25year', '26', '261', '275', '28', '280', '29', '295', '2nd', '30', '300', '31', '312', '32', '33', '330', '346', '35', '350', '37', '38', '386', '39', '3a', '3d', '3pointers', '40', '400', '401', '41', '42', '427', '43', '44', '45', '453', '47', '475', '48', '48volt', '494', '495', '4day', '4step', '50', '500', '51', '52', '

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
# list of text documents


# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(content_)
# summarize
# print(vectorizer.vocabulary_)
# print(vectorizer.idf_)
# encode document
#vector = vectorizer.transform([text[0]])
# summarize encoded vector
#print(vector.shape)
#print(vector.toarray())

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [16]:
vector = vectorizer.transform([content_[1]])
# summarize encoded vector
print(vector.shape)
print(vector.toarray())

(1, 8408)
[[0. 0. 0. ... 0. 0. 0.]]


# Bag-of-Words with Keras
The Keras Python library for deep learning also provides tools for encoding text using the bag-of words-model in the Tokenizer class.
As above, the encoder must be trained on source documents and then can be used to encode training data, test data and any other data in the future. The API also has the benefit of performing basic tokenization prior to encoding the words.


In [17]:
from keras.preprocessing.text import Tokenizer
tk = Tokenizer(num_words=2)
texts = content_
tk.fit_on_texts(texts)
# # create the tokenizer
# t = Tokenizer()
# # fit the tokenizer on the documents
# t.fit_on_texts(content_)
# # summarize what was learned
# # print(t.word_counts)
# # print(t.document_count)
# # print(t.word_index)
# # print(t.word_docs)
# # integer encode documents

In [18]:
tk.texts_to_sequences(texts)

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [1],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [1],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [1],
 [],
 [],
 [],
 [],
 [],
 [],
 [1],
 [],
 [],
 [],
 [],
 [],
 [],
 [1],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [1],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [1],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [1],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [1],
 [],
 [],
 [1],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [1],
 [1],
 [1],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [

In [19]:
tk.word_index

{'trump': 1,
 'new': 2,
 'dealbook': 3,
 'briefing': 4,
 'us': 5,
 'tech': 6,
 'trade': 7,
 'china': 8,
 'politics': 9,
 'says': 10,
 'facebook': 11,
 'deal': 12,
 'billion': 13,
 'may': 14,
 'big': 15,
 'trumps': 16,
 'tax': 17,
 'amazon': 18,
 'could': 19,
 'google': 20,
 'war': 21,
 '2020': 22,
 'house': 23,
 'news': 24,
 'plan': 25,
 'week': 26,
 'apple': 27,
 'million': 28,
 'fox': 29,
 'investors': 30,
 'wall': 31,
 'democrats': 32,
 'tariffs': 33,
 'get': 34,
 'back': 35,
 'uber': 36,
 'bank': 37,
 'chief': 38,
 'chinas': 39,
 'economy': 40,
 'plans': 41,
 'heres': 42,
 'business': 43,
 'stock': 44,
 'time': 45,
 'one': 46,
 'york': 47,
 'data': 48,
 'stocks': 49,
 'ceo': 50,
 'tesla': 51,
 'race': 52,
 'fed': 53,
 'first': 54,
 'europe': 55,
 'media': 56,
 'chinese': 57,
 'market': 58,
 'said': 59,
 'executive': 60,
 'fight': 61,
 'bid': 62,
 'pay': 63,
 'money': 64,
 'company': 65,
 'case': 66,
 'markets': 67,
 'go': 68,
 'online': 69,
 'report': 70,
 'wants': 71,
 'workers': 

In [20]:
encoded_docs = tk.texts_to_matrix(texts, mode='count')
print(encoded_docs)

[[0. 0.]
 [0. 0.]
 [0. 0.]
 ...
 [0. 0.]
 [0. 0.]
 [0. 0.]]


In [21]:
print(len(encoded_docs))
# print(len(text[i]))

4910


# Classification Models

In [22]:
train_data_features = vectorizer.fit_transform(content_)

In [28]:
#logisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

accuracy 0.3795180722891566
             precision    recall  f1-score   support

        Buy       0.37      0.40      0.39       340
       Hold       0.44      0.45      0.45       392
       Sell       0.28      0.25      0.26       264

avg / total       0.38      0.38      0.38       996



In [29]:
#RandomForest

from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

r_forests = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', RandomForestClassifier(n_estimators=100)),
               ])
r_forests.fit(X_train, y_train)

y_pred = r_forests.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

/Users/mollyperlich/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


accuracy 0.3795180722891566
             precision    recall  f1-score   support

        Buy       0.35      0.27      0.31       340
       Hold       0.41      0.62      0.49       392
       Sell       0.31      0.16      0.21       264

avg / total       0.36      0.38      0.35       996



In [30]:
#Keras 

import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

train_size = int(len(stock_df) * .7)
train_posts = stock_df['title'][:train_size]
train_tags = stock_df['label'][:train_size]

test_posts = stock_df['title'][train_size:]
test_tags = stock_df['label'][train_size:]

max_words = 1000
tokenize = text.Tokenizer(num_words=max_words, char_level=False)
tokenize.fit_on_texts(train_posts) # only fit on train

x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

num_classes = np.max(y_train) + 1
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

batch_size = 32
epochs = 4

# Build the model
model = Sequential()
model.add(Dense(512, input_shape=(max_words,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
              
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 3137 samples, validate on 349 samples
Epoch 1/4
3137/3137 [==============================] - 1s 279us/step - loss: 1.0888 - acc: 0.3726 - val_loss: 1.2078 - val_acc: 0.1834
Epoch 2/4
3137/3137 [==============================] - 1s 191us/step - loss: 0.9918 - acc: 0.5493 - val_loss: 1.2533 - val_acc: 0.2092
Epoch 3/4
3137/3137 [==============================] - 1s 199us/step - loss: 0.8855 - acc: 0.6321 - val_loss: 1.2817 - val_acc: 0.2751
Epoch 4/4
3137/3137 [==============================] - 1s 194us/step - loss: 0.7732 - acc: 0.6803 - val_loss: 1.3824 - val_acc: 0.2665


In [31]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test accuracy:', score[1])

1494/1494 [==============================] - 0s 38us/step
Test accuracy: 0.4049531459967934
